In [20]:
from time import sleep
import pydbus
      
import serial
import time
import re
import pandas as pd
import numpy as np
import pickle
import json

from IPython.display import IFrame
import serial.tools.list_ports

import pickle
import numpy as np
from pyargus.directionEstimation import *

import math
from collections import deque
from scipy.spatial import distance

from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.ticker import MultipleLocator

import pyaudio
import numpy as np
import matplotlib.pyplot as plt
import time
import wave
import IPython.display as ipd
import librosa.display
import sklearn
from scipy.stats import kurtosis, skew
import os

import csv

import warnings
warnings.filterwarnings('ignore')

%run audio.ipynb
%run featurizer.ipynb
%run location.ipynb
%run utils.ipynb
%run direction.ipynb

In [21]:
path = 'data/feature.csv'
runtime = 2 #sec

start_idx = 3
end_idx = 6

f = Featurizer()

f.set_utils(location_id='Arena', user_id='user_01')

for i in range(start_idx, end_idx):

    metadata = f.get_metadata()
    coord=  f.get_location()
    direction=  f.get_direction()
    audio = f.get_audio_features(i, runtime)   
    row = pd.concat([metadata, audio, coord, direction], axis=1)
    
    df = f.load_data(path)
    df = pd.concat([df, row], axis=0)
    df.to_csv(path, index=False)
    
    print("\n\n----------------------------")
    
f.close_connections()

Received location:     coordinate_x  coordinate_y  coordinate_z
0          1.75          4.78         -0.96
Received direction:    direction
0        NE
Recording starts..
Recording done!
Saved as data/recordings/record_0003.wav
Received location:     coordinate_x  coordinate_y  coordinate_z
0          1.76          4.77         -0.98
Received direction:    direction
0        NE
Recording starts..
Recording done!
Saved as data/recordings/record_0004.wav
Received location:     coordinate_x  coordinate_y  coordinate_z
0          1.76          4.77         -0.95
Received direction:    direction
0        NE
Recording starts..
Recording done!
Saved as data/recordings/record_0005.wav
